<a href="https://colab.research.google.com/github/derekmok/machine-vision-coursework/blob/main/Machine_Vision_Final_Lab_Model_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===== INSTALL DEPENDENCIES =====
!git init .
!git remote add origin https://github.com/derekmok/machine-vision-coursework.git
!git pull origin main

!pip install -r requirements.txt

In [1]:
# Import the required libraries
import torch
from torch.utils.data import DataLoader
from huggingface_hub import hf_hub_download
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import os
from tqdm.auto import tqdm
import time

# Please double, triple, quadruple check that the below code runs without errors before submitting.

## TODO 1 - Enter your HuggingFace username below:

In [2]:
hf_username = "derekmok"

## TODO 2 - Define your model EXACTLY as you did in your training code (otherwise there will be errors, and, possibly, tears).

Note below the classname is 'YourModelArchitecture'. That's because it literally needs to be YOUR MODEL ARCHITECTURE. This class definition is later referred to below in the 'load_model_from_hub' method. The architecture must match here, or it will not be able to instantiate the model weights correctly once it downloads them from HuggingFace. Pay very close attention to getting this right, please.

Replace the below code, and replace the corresponding line in the 'load_model_from_hub' method.

In [3]:
# =============================================================================
# 1. MODEL DEFINITION (must match training)
# =============================================================================
from neural_net.temporal_conv_net import TCNPushUpCounter
from neural_net.ensemble_model import EnsembleModel

## Download the test data from s3, and create the corresponding dataset + dataloader.

There's no TODO for you here. This text is just here to explain to you what this code does.

In this instance, the test data IS the training data you were provided in the Model Training notebook. This is by design. You do not have access to the test data. This is a simple check to make sure the mechanics of this notebook work.

You should achieve the same accuracy here in this notebook, as you did in your previous notebook (random seed notwithstanding).

In [4]:
# =============================================================================
# DOWNLOAD TEST DATA FROM S3
# =============================================================================

def download_test_data(bucket_name='training-and-validation-data',download_dir='./test-data'):
    s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

    bucket_name = 'prism-mvta'
    prefix = 'training-and-validation-data/'

    os.makedirs(download_dir, exist_ok=True)

    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

    video_names = []

    for page in pages:
        if 'Contents' not in page:
            print("No files found at the specified path!")
            break

        print("Downloading test data:\n")
        for obj in tqdm(page['Contents']):
            key = obj['Key']
            filename = os.path.basename(key)

            if not filename:
                continue

            video_names.append(filename)
            local_path = os.path.join(download_dir, filename)
            # print(f"Downloading: {filename}")
            s3.download_file(bucket_name, key, local_path)

    print(f"\nDownloaded {len(video_names)} test videos")
    return download_dir


# ============================================================================= # DATASET AND DATALOADER =============================================================================

from neural_net.data_loader import VideoDataset

## TODO 3 - Download your model from HuggingFace and instantiate it

Replace line 8 of the below code. Line 8 is where you instantiate YOUR MODEL ARCHITECTURE (which you re-defined above) with the weights you download from HuggingFace. Make sure you get the class name, and the arguments to the __init__ method correct.


This code just downloads the same model which you uploaded in the last notebook.

In [5]:
# =============================================================================
# DOWNLOAD MODEL FROM HUGGING FACE
# =============================================================================

def load_model_from_hub(repo_id, num_classes=10):
    model_path = hf_hub_download(repo_id=repo_id, filename="model.pt")

    model = EnsembleModel([TCNPushUpCounter(input_channels=6) for _ in range(5)])
    model.load_state_dict(torch.load(model_path, map_location='cpu'))

    print(f"Model loaded from {repo_id}")
    return model

model = load_model_from_hub(f"{hf_username}/mv-final-assignment", num_classes=10)

Model loaded from derekmok/mv-final-assignment


## TODO 4

Make sure the below code correctly evaluates your model performance on the given data!

This is your last chance to verify this before submission.

In [6]:
def evaluate(model, test_loader, dataset, device):
    model.eval()
    correct = 0
    total = 0

    all_preds = []
    all_labels = []
    all_times = []

    print("\n")

    with torch.no_grad():
        for idx, (frames, labels) in enumerate(test_loader):
            frames, labels = frames.to(device), labels.to(device)

            # Time the forward pass
            start_time = time.time()
            outputs = model(frames)
            if device.type == 'cuda':
                torch.cuda.synchronize()  # wait for GPU to finish
            end_time = time.time()

            inference_time = (end_time - start_time) * 1000  # ms
            all_times.append(inference_time)

            # For regression: model returns (count, density_map) tuple
            # Extract the count prediction (first element) and round to integer
            count_predictions = outputs[0]  # shape: [batch, 1]
            preds = torch.round(count_predictions.squeeze(-1)).long()  # shape: [batch]

            for i in range(labels.size(0)):
                batch_idx = idx * test_loader.batch_size + i
                video_name = dataset.video_files[batch_idx]
                pred = preds[i].item()
                true_label = labels[i].item()
                is_correct = "✓" if pred == true_label else "✗"

                print(f"{is_correct}  pred={pred}  true={true_label}  |  {inference_time:>7.1f}ms  |  {video_name}")

            correct += preds.eq(labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total
    return accuracy, all_preds, all_labels, all_times


# =============================================================================
# RUN INFERENCE
# =============================================================================

def run_inference(model, bucket_name='training-and-validation-data'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Download test data
    test_dir = download_test_data(bucket_name, './test-data')

    model = model.to(device)

    # Create dataloader
    test_dataset = VideoDataset.for_inference(test_dir)
    test_loader = DataLoader(
        test_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=0
    )

    print(f"\nRunning inference on {len(test_dataset)} test videos...")

    # Warmup (optional, helps get consistent GPU timings)
    if device.type == 'cuda':
        dummy = torch.randn(1, 1000, 6).to(device)
        with torch.no_grad():
            _ = model(dummy)
        torch.cuda.synchronize()

    total_start = time.time()
    accuracy, preds, labels, times = evaluate(model, test_loader, test_dataset, device)
    total_end = time.time()

    # Summary
    num_correct = sum(p == l for p, l in zip(preds, labels))
    num_wrong = len(preds) - num_correct

    print("\n" + "="*50)
    print("SUMMARY")
    print("="*50)
    print(f"Total videos:         {len(preds)}")
    print(f"Correct:              {num_correct}")
    print(f"Incorrect:                {num_wrong}")
    print(f"")
    print(f"ACCURACY:             {accuracy*100:.2f}%")
    print(f"")
    print(f"Total time:           {total_end - total_start:.2f}s")
    print(f"Avg per video:        {sum(times) / len(times):.1f}ms")
    print(f"Min latency:          {min(times):.1f}ms")
    print(f"Max latency:          {max(times):.1f}ms")
    print("="*50)
    return accuracy, preds, labels

_, _, _ = run_inference(model)

Using device: cpu



  0%|          | 0/77 [00:00<?, ?it/s]


Downloaded 77 test videos

Running inference on 77 test videos...




I0000 00:00:1767788432.370745 28598074 gl_context.cc:407] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1767788432.558565 28598076 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1767788432.642218 28598079 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1767788432.763307 28598082 landmark_projection_calculator.cc:81] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Detecting poses for ./test-data/2_sadfasjldkfjaseifj.mp4
✓  pred=2  true=2  |     30.9ms  |  2_sadfasjldkfjaseifj.mp4


...

✓  pred=2  true=2  |      3.7ms  |  2_sdlkjsaelijfksdjf.mp4
Detecting poses for ./test-data/3_dkk873lkjlksajdf.mp4


I0000 00:00:1767789584.714330 28615545 gl_context.cc:407] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
W0000 00:00:1767789584.789103 28615546 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1767789584.811443 28615546 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✓  pred=3  true=3  |      3.4ms  |  3_dkk873lkjlksajdf.mp4
Detecting poses for ./test-data/4_kling_20251209_Text_to_Video_Generate_a_452_1.mp4


I0000 00:00:1767789603.718586 28615963 gl_context.cc:407] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
W0000 00:00:1767789603.794460 28615965 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1767789603.819141 28615966 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✓  pred=4  true=4  |      6.0ms  |  4_kling_20251209_Text_to_Video_Generate_a_452_1.mp4

SUMMARY
Total videos:         77
Correct:              74
Incorrect:                3

ACCURACY:             96.10%

Total time:           1193.24s
Avg per video:        4.3ms
Min latency:          2.1ms
Max latency:          41.5ms
